In [1]:
import pandas as pd
import numpy as np
from functools import lru_cache
import ssl
import certifi
import math

In [2]:
# Set the CA bundle file
ssl._create_default_https_context = ssl._create_unverified_context
ssl._create_default_https_context().load_verify_locations(certifi.where())

@lru_cache(maxsize=None)  
def fetch_data_from_source(file):
    '''
    Files:
        f1sim-data-2022.parquet
        f1sim-data-2023.parquet
        
        f1sim-data-dictionary.xlsx
        
        f1sim-ref-left.csv
        f1sim-ref-line.csv
        f1sim-ref-right.csv
        f1sim-ref-turns.csv
    '''
    if file.split(".")[1] == "parquet":
        df = pd.read_parquet(f"https://data3001-racing.s3.ap-southeast-2.amazonaws.com/{file}")
    elif file.split(".")[1] == "csv":
        df = pd.read_csv(f"https://data3001-racing.s3.ap-southeast-2.amazonaws.com/{file}")
    elif file.split(".")[1] == "xlsx":
        df = pd.read_excel(f"https://data3001-racing.s3.ap-southeast-2.amazonaws.com/{file}") 
    return df

In [3]:
# Load data files
f1_2022_df = fetch_data_from_source('f1sim-data-2022.parquet')
f1_2023_df = fetch_data_from_source("f1sim-data-2023.parquet")
ref_left = fetch_data_from_source("f1sim-ref-left.csv")
ref_line = fetch_data_from_source("f1sim-ref-line.csv")
ref_right = fetch_data_from_source("f1sim-ref-right.csv")
ref_turns = fetch_data_from_source("f1sim-ref-turns.csv")

In [9]:
def calc_acceleration(data):
    # distance in metres
    distances = []

    # time in milliseconds
    times = []

    # speed in meters per milliseconds
    speeds = []

    # acceleration is in meters per milliseconds per millisecond
    accelerations = []

    for count, row in data.iterrows():
        sessionid1 = row['SESSION_IDENTIFIER']
        lapid1 = row['LAP_NUM']
        time1 = row['CURRENT_LAP_TIME_MS']
        x1 = row['WORLDPOSX']
        y1 = row['WORLDPOSY']
        try:
            sessionid2 = f1_2022_df['SESSION_IDENTIFIER'].iloc[count-1]
            lapid2 = f1_2022_df['LAP_NUM'].iloc[count-1]
            time2 = f1_2022_df['CURRENT_LAP_TIME_MS'].iloc[count-1]
            x2 = f1_2022_df['WORLDPOSX'].iloc[count-1]
            y2 = f1_2022_df['WORLDPOSY'].iloc[count-1]
            if sessionid1 == sessionid2 and lapid1 == lapid2:
                distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
                distances.append(distance)
                
                time = time1 - time2
                times.append(time)
                
                if time != 0:
                    speed = distance / time
                else:
                    speed = np.nan    
                
                speeds.append(speed)
                
                if time != 0:
                    acceleration = (speed - speeds[count - 1]) / time 
                else:
                    acceleration = np.nan
                
                accelerations.append(acceleration)
            else:
                distances.append(0)
                times.append(0)
                speeds.append(0)
                
        except KeyError:
            distances.append(np.nan)
            times.append(np.nan)
            speeds.append(np.nan)